In [1]:
!pip install keras
!pip install tensorflow
!pip install gensim

    100% |████████████████████████████████| 307kB 2.1MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 7.0MB/s eta 0:00:01
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 49.1MB 16kB/s  eta 0:00:01 0% |▎                               | 399kB 2.4MB/s eta 0:00:20    1% |▍                               | 552kB 3.2MB/s eta 0:00:16    3% |█                               | 1.7MB 6.1MB/s eta 0:00:08    15% |████▉                           | 7.4MB 8.6MB/s eta 0:00:05    44% |██████████████                  | 21.6MB 14.2MB/s eta 0:00:02    56% |██████████████████              | 27.5MB 4.2MB/s eta 0:00:06    60% |███████████████████▍            | 29.8MB 7.1MB/s eta 0:00:03    70% |██████████████████████▌         | 34.4MB 9.9MB/s eta 0:00:02    73% |███████████████████████▍        | 35.9MB 7.8MB/s eta 0:00:02�▊     | 41.0MB 16.3MB/s eta 0:00:0

    100% |████████████████████████████████| 22.6MB 40kB/s  eta 0:00:01 2% |▊                               | 481kB 5.4MB/s eta 0:00:05    16% |█████▏                          | 3.7MB 10.0MB/s eta 0:00:02    17% |█████▊                          | 4.0MB 9.7MB/s eta 0:00:02    69% |██████████████████████▏         | 15.6MB 4.8MB/s eta 0:00:02    72% |███████████████████████▎        | 16.4MB 7.0MB/s eta 0:00:01    80% |█████████████████████████▋      | 18.1MB 8.6MB/s eta 0:00:01
    100% |████████████████████████████████| 1.4MB 302kB/s ta 0:00:011    96% |███████████████████████████████ | 1.3MB 611kB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 2.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 4.9MB/s eta 0:00:01
    100% |████████████████████████████████| 4.3MB 208kB/s eta 0:00:01    31% |██████████▎                     | 1.4MB 5.5MB/s eta 0:00:01    87% |████████████████████████████    | 3.8MB 6.0MB/s eta 0:00:01
    99% |██████████████████████████████

In [2]:
import numpy as np
np.random.seed(13)

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import gensim

Using TensorFlow backend.


In [3]:
path=get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
corpus = open(path).readlines()[:300]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2


180224/173595 [===============================] - 0s 2us/step


In [4]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)

            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)

In [5]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

In [6]:
cbow.layers

In [7]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [8]:
corpus[:5]

[[348, 349, 65, 115, 10, 116, 57, 196, 197],
 [22, 117, 66, 17, 1, 67, 8, 350, 351, 29, 38, 352, 4, 27],
 [146, 38, 353, 354, 16, 355, 356, 5, 198, 5, 147, 32],
 [357, 67, 5, 118, 1, 358, 8, 1, 199, 148, 359, 360],
 [27, 22, 117, 32, 361, 29, 362, 148, 363]]

In [9]:
for ite in range(10):
    loss = 0.
    for i, (x, y) in enumerate(generate_data(corpus, window_size, V)):
        if i<4:
            print(' ', i, x, y.shape)
        loss += cbow.train_on_batch(x, y)
    print(i)

    print(ite, loss)

  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
0 17412.565080165863
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
1 16152.256516456604
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
2 16024.155794858932
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
3 15933.206165194511
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
4 15825.334559202194
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
5 15722.699430406094
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 1

In [10]:
f = open('vectors-cbow.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

8

In [11]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [12]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors-cbow.txt', binary=False)

In [13]:
w2v.most_similar(positive=['the'])

[('a', 0.6486455202102661),
 ('one', 0.6459208130836487),
 ('this', 0.5973585844039917),
 ('adventures', 0.5596011281013489),
 ('alice’s', 0.5594749450683594),
 ('any', 0.5505132675170898),
 ('those', 0.5317860245704651),
 ('miles', 0.527005136013031),
 ('help', 0.5243159532546997),
 ('no', 0.5233829021453857)]

In [14]:
w2v.most_similar(positive=['alice'])

[('she', 0.610198974609375),
 ('you', 0.6003904342651367),
 ('poor', 0.5682430863380432),
 ('them', 0.5612325668334961),
 ('that', 0.5536919832229614),
 ('eat', 0.5531008839607239),
 ('now', 0.5518883466720581),
 ('dark', 0.5485798120498657),
 ('marked', 0.548270046710968),
 ('thought', 0.5456354022026062)]